<a href="https://colab.research.google.com/github/rubabshz/Rubab_INFO5731_Spring2020/blob/main/Rubab_INFO5731_Assignment_Three.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment.

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as all the 100 reviews (abstracts, or tweets). 


In [54]:
#1.1 Tri-grams frequency
import nltk
nltk.download('punkt')
with open("/content/CleanData.csv", "r", encoding='utf-8') as file:
     data = file.read()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [57]:
import pandas
from textblob import TextBlob
tri_grams=TextBlob(data).ngrams(3)
hist=dict()
for gram in tri_grams:
 gram=tuple(gram)
 hist[gram]=hist.get(gram,0)+1
pandas.DataFrame.from_dict(hist, orient = 'index', columns=['tri-grams frequency'])

,tri-grams frequency
"(Cleaned, b, march)",1
"(b, march, pm)",7
"(march, pm, edt)",7
"(pm, edt, real)",7
"(edt, real, time)",7
...,...
"(is, b, the)",1
"(http, co, xsuiyyml)",1
"(co, xsuiyyml, b)",1
"(xsuiyyml, b, the)",1


In [58]:
#1.2 Probabilities of Bi-grams

import pandas
monty = TextBlob (data)
bi_grams=TextBlob(data).ngrams(2)
# i t(bi )
#print(bi_grams)

# make a dict of bigrams and their frequency
hist={}
for gram in bi_grams:
    gram=tuple(gram)
    hist[gram]=hist.get(gram,0)+1
#   calculate the probabilities of bigrams
bigr_prob={}
for key, val in hist.items():
    bigr_prob[key]=val/monty.words.count(key[0])

df = pandas.DataFrame.from_dict(bigr_prob, orient = 'index', columns=['bigrams probabilities'])
df['bigram frequency']=[val for val in hist.values()]
df.head(10)

#print(bi_grams)

,bigrams probabilities,bigram frequency
"(Cleaned, b)",1.000000,1
"(b, march)",0.090909,7
"(march, pm)",0.777778,7
"(pm, edt)",1.000000,7
"(edt, real)",1.000000,7
"(real, time)",1.000000,7
"(time, mapping)",0.875000,7
"(mapping, wuhancoronavirus)",1.000000,7
"(wuhancoronavirus, ncov)",0.269231,7
"(ncov, global)",1.000000,7


In [62]:
#1.3 Extract all Noun phrases
import pandas
import nltk
nltk.download('brown')
monty=TextBlob(data)
# make a dictionary of noun phrases and their frequency in the whole corpus
hist1={}
for np in monty.noun_phrases:
 hist1[np]=hist1.get(np,0)+1
all_nphrases= [val for val in hist1.values()]

# find the maximum frequency of these noun phrases in the whole dataset
max_refreq = max([val for val in hist1.values()])

# calculate the relative probability of reviews to others based on noun phrases
re_prob=[]
for line in data.splitlines():
 monty=TextBlob(line)
 nphrases= monty.noun_phrases
 hist2={} # make a dictionary of noun phrases and their frequency in each review
 for np in nphrases:
    hist2[np]=hist2.get(np,0)+1
 for key, val in hist2.items():
    hist2[key] = val/max_refreq
 re_prob.append(hist2)

doc_index=[n for n in range(1, len(re_prob)+1)]

df=pandas.DataFrame (re_prob, index=doc_index)
df.fillna(0, inplace=True) # set Null as 0
df

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


,cleaned,b march pm edt,real time,wuhancoronavirus ncov,global case,case and\xe \x \xa http co twyj zodss,rest wuhancoronavirus vaccination timeline,de\xe \x \xa http co lwulsrpm w,wuhancoronavirus http co ipcmob cvw,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co sh q zjq,b state department lead investigator,lab \n\nhttps co eotcqetzj \n\nvia\xe \x \xa http co m q uqffjt,b tedros,\n\xe \x c\x \xe \xac\x \nhttps co nkacos ld \n\ncoronaviruschina pndahugger\ncoronachan coronachan \xe \x \xb \xe \x \xad\xe \x \x a\xe \x \xa \xe \x \x \xe \x \x \xe \x \xa http co m cyhc,b aymanm timobrien,china virus problem asian,problem don\xe \x \x mix \ni asia\xe \x \xa http co lb ig s u,problem don\xe \x \x mix \ni asian,it\xe \x \xa http co i qjhycep,b timobrien,i\xe \x \xa http co e z vlwdr,b mmpadellan ok the\nwuhancoronavirus\nwuflu\nkungflu\nwuhanchinesecoronavirus,b gordongchang,hong kong world \n\nccpchina,accountable \n\n hongkong \n \xe \x \xa http co okaq boanq,b trump accomplishment term \n\nadd nobel peace prize nomination,actual accomplishment incl promot\xe \x \xa http co ia swlzq,case and\xe \x \xa http co wiyoh,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co s kn bav,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co ycl ygfjpr,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co bn ruk gz,\n\xe \x c\x \xe \xac\x \nhttps co nkacos ld \ncoronaviruschina pndahugger\ncoronachan coronachan \xe \x \xb \xe \x \xad\xe \x \x a\xe \x \xa \xe \x \x \xe \x \x \xe \x \xa http co yjelptq br,b globaltimesnews covid impact thanks dirty wet market wuhan\n\nwuhanvirus wuhancoronavirus china\xe \x \xa http co az twho,b sharpiedj michaelpsenger,ccpvirus wuhancoronavirus past,b india \xf \x f\x \xae\xf \x f\x \xb example,vaccine country \nwuhancoronavirus pandemic,world \xf \x f\x c\x \n we\xe \x \xa http co qm wicxtwf,b mattwridley wuhancoronavirus,world thanks,...,b houseforeigngop,ccp ccp co\xe \x \xa http co nl zqoxpvh,b senrickscott secblinken jakesullivan,ccp enemy democracy \nplease,wall street help\xe \x \xa http co nqtcwfhrqo,b jenvanlaar,crazy virus fundamentalist \nthey,wuhancoronavirus\xe \x \xa http co rkhe zvstr,case and\xe \x \xa http co ibamyvkxzt,b ferric memorial day weekend life return,i\xe \x \xa http co phe ebzqcz,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co hz r zvdp,b reeb matthew toneshyus xandesanctis varadmehta trump,fine lockdown,t\xe \x \xa http co ftjwhucint,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co oqjfjxehyv,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe \xaf\x \xe \xad\xa \xe \xb \x \xe \xae\x e\xe \xaa\x c\xe \xae\xa \n laborunfall sarscov sarscov virus pandemie\n\xe \x \xab\xe \xb \xbd\xe \xa \xa drlimengyan\n\xc \xa \xe \x \xa http co hn esbaavp,truth covid \xef\xbc\x \n\xe \xb \xad\xe \x \xb \xe \x \x \xe 

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [63]:
#2.1 Documents-terms weights matrix bold text

import pandas
from sklearn.feature_extraction.text import TfidfVectorizer
doc=[doc for doc in data.splitlines()]
corpus = {v: k for v, k in enumerate(data.splitlines(),1)}
tfidf=TfidfVectorizer(input=doc, max_features=50, min_df=0.2, lowercase=True, analyzer='word', stop_words='english', ngram_range = (1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)

df



,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
covid,0.0,0.000000,0.000000,0.000000,0.081217,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.081217,0.081217,0.081217,0.000000,0.528946,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.607465,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081217,0.000000,0.081217,0.081217,0.081217,0.081217,0.000000,0.000000,0.000000,0.081217,0.082691,0.081217,0.082691,0.081217,0.081217,0.081217,0.081217,0.081217,0.081217,0.081217,0.081217
http,0.0,0.327066,0.327066,0.491557,0.042619,0.401297,0.076016,0.284915,0.284915,0.284915,0.0,0.401297,0.401297,0.327066,0.042619,0.042619,0.042619,0.076016,0.277566,0.0,0.277190,0.327066,0.327066,0.401297,0.000000,0.327066,0.327066,0.327066,0.401297,0.0,0.0,0.327066,0.401297,0.401297,0.318769,0.327066,0.076016,0.0,0.327066,0.401297,0.401297,0.491557,0.401297,0.401297,0.0,0.076016,0.401297,0.327066,0.000000,0.0,0.0,0.327066,0.327066,0.401297,0.284214,0.327066,0.328957,0.042619,0.401297,0.042619,0.042619,0.042619,0.042619,0.401297,0.075862,0.284915,0.042619,0.043392,0.042619,0.043392,0.042619,0.042619,0.042619,0.042619,0.042619,0.042619,0.042619,0.042619
laborunfall,0.0,0.000000,0.000000,0.000000,0.084442,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084442,0.084442,0.084442,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084442,0.000000,0.084442,0.084442,0.084442,0.084442,0.000000,0.000000,0.000000,0.084442,0.085975,0.084442,0.085975,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442
laborunfall sarscov,0.0,0.000000,0.000000,0.000000,0.084442,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084442,0.084442,0.084442,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084442,0.000000,0.084442,0.084442,0.084442,0.084442,0.000000,0.000000,0.000000,0.084442,0.085975,0.084442,0.085975,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442
laborunfall sarscov sarscov,0.0,0.000000,0.000000,0.000000,0.084442,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084442,0.084442,0.084442,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084442,0.000000,0.084442,0.084442,0.084442,0.084442,0.000000,0.000000,0.000000,0.084442,0.085975,0.084442,0.085975,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442,0.084442
pandemie,0.0,0.000000,0.000000,0.000000,0.084442,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084442,0.084442,0.084442,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0

In [64]:
#2.2 Rank the documents
# design a query
from nltk.corpus import stopwords
import nltk
import re
from nltk.stem import PorterStemmer
nltk.download('stopwords')
nltk.download('wordnet')
from textblob import Word
query="Originated in Wuhan, covid-19 also known as the coronavirus"

# preprocess query (lowercase, remove punctuations, remove stopwords, stem and lematize)
query=query.lower()
query=re.sub('[^\w\s]', '', query)
stop=stopwords.words('english')
query= " ".join(x for x in query.split() if x not in stop)
st=PorterStemmer()
query=" ".join([st.stem(word) for word in query.split()])
query= " ".join([Word(word).lemmatize() for word in query.split()])
print(query)

# append query to the working corpus (all cleaned reviews)
with open("COVIDtweets.txt", "a", encoding='utf-8') as file:
     file.write(data)
     file.write(query)
with open("COVIDtweets.txt", "r", encoding='utf-8') as file:
     data_query = file.read()


# Caculate tfidf, and extract features representing all reviews and the query
doc=[doc for doc in data_query.splitlines()]
corpus = {v: k for v, k in enumerate(data_query.splitlines(),1)}
tfidf=TfidfVectorizer(input=doc, max_features=50, min_df=0.2, lowercase=True, analyzer='word', stop_words='english', ngram_range = (1,3))
tfs=tfidf.fit_transform(corpus.values())
feature_names= tfidf.get_feature_names()
corpus_index=[n for n in corpus]
df=pandas.DataFrame(tfs.T.todense(), index=feature_names, columns=corpus_index)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
origin wuhan covid19 also known coronaviru


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
covid,0.0,0.000000,0.000000,0.000000,0.081052,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.081052,0.081052,0.081052,0.000000,0.527632,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.605559,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.081052,0.000000,0.081052,0.081052,0.081052,0.081052,0.000000,0.000000,0.000000,0.081052,0.082526,0.081052,0.082526,0.081052,0.081052,0.081052,0.081052,0.081052,0.081052,0.081052,0.081052,0.0
http,0.0,0.327623,0.327623,0.493319,0.042752,0.401372,0.076189,0.285274,0.285274,0.285274,0.0,0.401372,0.401372,0.327623,0.042752,0.042752,0.042752,0.076189,0.278307,0.0,0.277935,0.327623,0.327623,0.401372,0.000000,0.327623,0.327623,0.327623,0.401372,0.0,0.0,0.327623,0.401372,0.401372,0.319412,0.327623,0.076189,0.0,0.327623,0.401372,0.401372,0.493319,0.401372,0.401372,0.0,0.076189,0.401372,0.327623,0.000000,0.0,0.0,0.327623,0.327623,0.401372,0.284902,0.327623,0.329495,0.042752,0.401372,0.042752,0.042752,0.042752,0.042752,0.401372,0.076202,0.285274,0.042752,0.043530,0.042752,0.043530,0.042752,0.042752,0.042752,0.042752,0.042752,0.042752,0.042752,0.042752,0.0
laborunfall,0.0,0.000000,0.000000,0.000000,0.084252,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084252,0.084252,0.084252,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084252,0.000000,0.084252,0.084252,0.084252,0.084252,0.000000,0.000000,0.000000,0.084252,0.085784,0.084252,0.085784,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.0
laborunfall sarscov,0.0,0.000000,0.000000,0.000000,0.084252,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084252,0.084252,0.084252,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084252,0.000000,0.084252,0.084252,0.084252,0.084252,0.000000,0.000000,0.000000,0.084252,0.085784,0.084252,0.085784,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.0
laborunfall sarscov sarscov,0.0,0.000000,0.000000,0.000000,0.084252,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084252,0.084252,0.084252,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.084252,0.000000,0.084252,0.084252,0.084252,0.084252,0.000000,0.000000,0.000000,0.084252,0.085784,0.084252,0.085784,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.084252,0.0
pandemie,0.0,0.000000,0.000000,0.000000,0.084252,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.084252,0.084252,0.084252,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000

In [65]:
#2.2.2 Cosine Similarity
import pandas
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
# looping over documents to calculate cosine similarity of query (doc77) to others
# make a dict of doc and results
i=1
quer_sim = {}
while i <=77:
      cosim=cosine_similarity(tfs[77], tfs[i]) # a function of sklearn
      quer_sim['doc'+str(i)] = cosim
      i+=1
# sort the results to rank the similarity of each doc to the query
quer_sim_rank = {key:rank for rank, key in enumerate(sorted (quer_sim, key=quer_sim.get, reverse=True),1)}
cosim_list = [quer_sim[doc]for doc in sorted(quer_sim)]
rankdf=pandas.DataFrame.from_dict (quer_sim_rank,orient = 'index', columns = ['ranking'])
rankdf ['cosine similarity'] = cosim_list
rankdf.head(20)

,ranking,cosine similarity
doc4,1,[[0.34589987941993766]]
doc14,2,[[0.0]]
doc15,3,[[0.4237620378030743]]
doc16,4,[[0.4237620378030743]]
doc57,5,[[0.34589987941993766]]
doc59,6,[[0.9999999999999999]]
doc60,7,[[0.9999999999999999]]
doc61,8,[[0.9999999999999999]]
doc62,9,[[0.6873260550928898]]
doc66,10,[[0.3365984141465217]]


In [66]:
# Calculate cosine similarity of query with documents manually
import math
import numpy as np

# define a function to calculate cosine similarity of two vectors
def cosim (v1,v2):
    x=v1.toarray()
    y=v2.toarray()
    return np.sum(x*y)/math.sqrt(np.sum(x*x)*np.sum(y*y))
# Using defined function to calculate cosine similarity of the query to each doc
# looping through each doc, calculate and store in a dict
i=1
query_sim = {}
while i <=77:
      cosin_similarity = cosim(tfs[77], tfs[i])
      cosin_similarity = str(cosin_similarity)
      query_sim['doc'+str(i)] = cosin_similarity
      i+=1
query_sim_noNul= {k:float(v) for k,v in query_sim.items() if v != 'nan'} # remove Null values from the dict
# sort the results to rank
sorted_query_sim = sorted (query_sim_noNul, key=query_sim_noNul.get, reverse=True)
query_sim_rank = {key:rank for rank, key in enumerate(sorted_query_sim,1)}
cosim_list = [query_sim_noNul[doc] for doc in sorted_query_sim]
rank_df=pandas.DataFrame.from_dict (query_sim_rank,orient = 'index', columns = ['ranking'])
rank_df ['cosine similarity'] = cosim_list
rank_df.head(100)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':


,ranking,cosine similarity
doc4,1,1.0
doc14,2,1.0
doc15,3,1.0
doc16,4,1.0
doc57,5,1.0
...,...,...
doc29,71,0.0
doc30,72,0.0
doc37,73,0.0
doc44,74,0.0


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each review (abstract or tweet) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [ ]:

# Link: https://github.com/rubabshz/Rubab_INFO5731_Spring2020/blob/main/CleanData.csv